# explorer
fill a playlist with all tracks or just top tracks from every artist specified from:
- a playlist
- artists user is following
- a single artist
- saved songs
- a genre

## references
- https://spotipy.readthedocs.io/en/latest/
- https://developer.spotify.com/documentation/general/guides/scopes/

In [1]:
import spotipy
import spotipy.util as util
import os
import sys
import pprint
import string
import traceback

In [69]:
SPOTIPY_CLIENT_ID='da0aa5c92e024de3827e9db0ea2019fd'
SPOTIPY_CLIENT_SECRET='8af549cae6224233b7609aea4963b641'
SPOTIPY_REDIRECT_URI='https://github.com/danielpersonius'
my_username = "Daniel Personius"

# SPOTIPY_CLIENT_ID     = 'client-id'
# SPOTIPY_CLIENT_SECRET = 'client-secret'
# SPOTIPY_REDIRECT_URI  = 'redirect-url'
# my_username = "my-username"

os.environ['SPOTIPY_CLIENT_ID']     = SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = SPOTIPY_CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI']  = SPOTIPY_REDIRECT_URI

# permissions
my_scope = 'user-library-read user-follow-read playlist-read-collaborative playlist-read-private playlist-modify-private playlist-modify-public'

token = util.prompt_for_user_token(
        username=my_username,
        scope=my_scope,
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET,
        redirect_uri=SPOTIPY_REDIRECT_URI)

In [70]:
if token: 
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    
else:
    print("fuck ur tkn")

In [39]:
sources = ['playlist', 'artists', 'artist', 'saved', 'genre']
source_type = input("Where do you want to get artists from?\n(1) playlist\n(2) followed artists\n(3) artist\n(4) saved songs\n(5) genre\n")
print(sources[int(source_type) - 1])

source = sources[int(source_type) - 1] 

if source == 'playlist':
    playlist_name = input('Which playlist?\n')
elif source == 'artist':
    artist_name = input('Which artist?\n')
elif source == 'genre':
    genre_name = input('Which genre?\n')
    
depths = ['all', 'top']
depth_type = input("How many tracks do you want from each?\n(1) All\n(2) Top 10\n")
depth = depths[int(depth_type) - 1]
print(depth)

Where do you want to get artists from?
(1) playlist
(2) followed artists
(3) artist
(4) saved songs
(5) genre
1
playlist
Which playlist?
n-brn
How many tracks do you want from each?
(1) All
(2) Top 10
2
top


In [119]:
# helper functions
def get_followed_artists(spot):
    '''
    Get all the artists the user follows
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all artists the user follows
    '''
    results = spot.current_user_followed_artists()
    all_artists = results['items']
    while results['next']:
        results = spot.next(results)
        all_artists.extend(results['items'])
    return all_artists

def get_playlists(spot):
    '''
    Get all the user's playlists
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all user's playlists
    '''
    results = spot.current_user_playlists()
    all_playlists = results['items']
    while results['next']:
        results = spot.next(results)
        all_playlists.extend(results['items'])
    return all_playlists

def get_playlist(spot, playlist_name):
    '''
    '''
    all_playlists = get_playlists(spot)
    for playlist in all_playlists:
        if playlist['name'] == playlist_name:
            return playlist
    return None

def get_playlist_artists(spot, user_id, playlist_id):
    '''
    Get all the artists that appear on a playlist
    
    Arguments:
        spot        - spotipy object
        user_id     - user id
        playlist_id - playlist id 
        
    Returns:
        list - all artists that appear
    '''
    results = spot.user_playlist_tracks(user_id, playlist_id)
    all_tracks = results['items']
    while results['next']:
        results = spot.next(results)
        all_tracks.extend(results['items'])
    
    all_artists = set()
    for track in all_tracks:
        for artist in track['track']['artists']:
            name = artist['name']
            id = artist['id']
            # getting weird occurence of an empty artist name, so filter out
            if name != '':
                all_artists.add((name, id))
    return all_artists

def get_artist_track_ids(spot, artist_id, depth):
    '''
    
    Arguments:
        
    Returns:
    
    Todo:
        no api call for 'all', so call spot.search(artist[0], type='track') and paginate
    '''
    all_tracks = []
    tracks = []
    if depth == 'top':
        all_tracks = spot.artist_top_tracks(artist_id)
        for track in all_tracks.items():
            print(type(track[0]))
            #tracks.append(track[1][0]['id'])
        return tracks
    return None

def get_saved_tracks(spot):
    '''
    Get all tracks the user has saved
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all saved songs
    '''
    results = spot.current_user_saved_tracks()
    all_saved_tracks = results['items']
    while results['next']:
        results = spot.next(results)
        all_saved_tracks.extend(results['items'])
    return all_saved_tracks

def get_genre_artists(sp, genre_name):
    '''
    '''
    return None

def create_playlist(spot, playlist_name, is_public, user_id):
    '''
    create a playlist
    
    Arugments:
        sp - Spotipy object
        playlist_name - name of playlist artists were gathered from
        is_public - whether playlist is visible or not
        user_id - the user for who the playlist is for
        
    Returns:
        void
        
    Todo:
        - conditional playlist name
    '''
    playlist_prefix = "explore "
    print(playlist_name)
    if get_playlist(spot, playlist_prefix + playlist_name) is None:
        #, description="Explore " + playlist_name
        spot.user_playlist_create(user=user_id, name=playlist_prefix + playlist_name, public=is_public)
    return get_playlist(spot, playlist_prefix + playlist_name)['id']

def add_tracks_to_playlist(spot, track_ids, playlist_id, user_id):
    '''
    '''
    spot.user_playlist_add_tracks(user=user_id, playlist_id=playlist_id, tracks=track_ids)

In [120]:
current_user = sp.me()
if source == 'playlist':
    playlist = get_playlist(sp, playlist_name)
    if playlist is not None:
        explore_playlist_id = create_playlist(sp, playlist_name, False, current_user['id'])
        playlist_created = 1
    playlist_artists = []
    playlist_artists = get_playlist_artists(sp, current_user['id'], playlist['id'])
    
    for artist in playlist_artists:
        artist_track_ids = get_artist_track_ids(sp, artist[1], depth)
        pprint.pprint(artist_track_ids)
#         add_tracks_to_playlist(sp, artist_track_ids, explore_playlist_id, current_user['id'])
elif source == 'artists':
    followed_artists = get_followed_artists(sp)
    
elif source == 'artist':
    artist_tracks = get_artist_tracks(sp, artist_name)
    
elif source == 'saved':
    saved_tracks = get_saved_tracks(sp)
    
elif source == 'genre':
    genre_artists = get_genre_artists(sp, genre_name)

n-brn
<class 'str'>
[]
<class 'str'>
[]
<class 'str'>
[]
<class 'str'>
[]
<class 'str'>
[]


In [101]:
for artist in playlist_artists:
    artist_track_ids = get_artist_track_ids(sp, artist, depth)
    pprint.pprint(artist_track_ids)

['3FuFfWcRmFkLtv7j3mQNY8']
['4Js1cgSrcGiTNCEwUOFhvv']
['4bmvfx88H75CcEE0mKXD3q']
['4Js1cgSrcGiTNCEwUOFhvv']
['7ipGvvE90ROUNFw7ZBXBTO']
['7ipGvvE90ROUNFw7ZBXBTO']
['2sKTYRCgUHXmIhI2hoBuZE']
